# **Homework**: Data talks club data engineering zoomcamp Data loading workshop





# 1. Use a generator

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

# Q1

In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

total =[]
for sqrt_value in generator:

    total.append(sqrt_value)
    print(sqrt_value)

print("Total",sum(total))


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
Total 8.382332347441762


# Q2

In [2]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

limit = 13
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)



1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




# Q3

In [3]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}
total_age = []
for person in people_1():
    total_age.append(person['Age'])
    print(person)
print('Total age for the first generator is:',sum(total_age))




def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}
total_age2 =[]
for person in people_2():
    total_age2.append(person['Age'])
    print(person)
print('Total age for the second generator is:',sum(total_age2))


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
Total age for the first generator is: 140
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}
Total age for the second generator is: 213


In [4]:
%%capture
!pip install dlt[duckdb] # Install dlt with all the necessary DuckDB dependencies

In [5]:
#loading geenerators to duck db
import dlt


generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')
info = generators_pipeline.run(people_1(),
										table_name="Customer",
										write_disposition="replace")

print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.25 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707847247.4641225 is LOADED and contains no failed jobs


In [6]:

import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n Customer table below:")

customers = conn.sql("SELECT * FROM Customer").df()

total_age = conn.sql("SELECT sum(age) FROM Customer").df

display(total_age)
display(customers)


Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ customer            │
└─────────────────────┘




 Customer table below:


<bound method PyCapsule.df of ┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      140 │
└──────────┘
>

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707847247.4641225,d6DySm6JqiyrBw,None
1,2,Person_2,27,City_A,1707847247.4641225,3DL7IuBArV7vSA,None
2,3,Person_3,28,City_A,1707847247.4641225,qNi/uplpMdXSgg,None
3,4,Person_4,29,City_A,1707847247.4641225,ifgwsS+8ViF1Lw,None
4,5,Person_5,30,City_A,1707847247.4641225,l0F1DFO/N65Z4A,None


In [7]:
info = generators_pipeline.run(people_2(),
										table_name="Customer",
										write_disposition="append")

print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.17 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707847248.2481518 is LOADED and contains no failed jobs


In [8]:
#Total customers
import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n Customer table below:")

customers = conn.sql("SELECT * FROM Customer").df()

total_age = conn.sql("SELECT sum(age) FROM Customer").df

display(total_age)
display(customers)


Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ customer            │
└─────────────────────┘




 Customer table below:


<bound method PyCapsule.df of ┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      353 │
└──────────┘
>

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707847247.4641225,d6DySm6JqiyrBw,None
1,2,Person_2,27,City_A,1707847247.4641225,3DL7IuBArV7vSA,None
2,3,Person_3,28,City_A,1707847247.4641225,qNi/uplpMdXSgg,None
3,4,Person_4,29,City_A,1707847247.4641225,ifgwsS+8ViF1Lw,None
4,5,Person_5,30,City_A,1707847247.4641225,l0F1DFO/N65Z4A,None
5,3,Person_3,33,City_B,1707847248.2481518,bDEMeLYH2O3EWQ,Job_3
6,4,Person_4,34,City_B,1707847248.2481518,SboG8mN6k2CTrA,Job_4
7,5,Person_5,35,City_B,1707847248.2481518,0lUveiAsSD1Gdg,Job_5
8,6,Person_6,36,City_B,1707847248.2481518,UihrllE09DEzzg,Job_6
9,7,Person_7,37,City_B,1707847248.2481518,ej1k4vehRuG4WQ,Job_7


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Q3

In [11]:
import dlt


generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='all_customers')
info = generators_pipeline.run(people_1(),
										table_name="Customers",
                    primary_key="id",
										write_disposition="replace")

print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.25 seconds
1 load package(s) were loaded to destination duckdb and into dataset all_customers
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707847671.3742323 is LOADED and contains no failed jobs


In [12]:
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='all_customers')
info = generators_pipeline.run(people_2(),
										table_name="Customers",
										write_disposition="merge",
                    primary_key="id")

print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.46 seconds
1 load package(s) were loaded to destination duckdb and into dataset all_customers
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707847733.4607875 is LOADED and contains no failed jobs


In [13]:

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n Customers table below:")

customers = conn.sql("SELECT * FROM Customers").df()

total_age = conn.sql("SELECT sum(age) FROM Customers").df

display(total_age)
display(customers)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ customers           │
└─────────────────────┘




 Customers table below:


<bound method PyCapsule.df of ┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      266 │
└──────────┘
>

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707847671.3742323,2VXsaBnjhHUDTw,None
1,2,Person_2,27,City_A,1707847671.3742323,MuKEZee0HsOqAw,None
2,5,Person_5,35,City_B,1707847733.4607875,3R+9CCGnfObigQ,Job_5
3,4,Person_4,34,City_B,1707847733.4607875,vd1rUBobMSWBxQ,Job_4
4,8,Person_8,38,City_B,1707847733.4607875,EfxLNy6Cg5PH+A,Job_8
5,7,Person_7,37,City_B,1707847733.4607875,3x5rwB/C1b25Jw,Job_7
6,3,Person_3,33,City_B,1707847733.4607875,TYjIe+ObAVpb0A,Job_3
7,6,Person_6,36,City_B,1707847733.4607875,FhOxgHKZ99XMBA,Job_6
